In [1]:
# 텐서플로의 그레이디언트 테이프
import tensorflow as tf
x = tf.Variable(0.)

In [3]:
with tf.GradientTape() as tape:
    y = 2*x+3
grad_of_y_wrt_x = tape.gradient(x,y)    

In [5]:
x = tf.Variable(tf.zeros(2,2))
with tf.GradientTape() as tape:
    y = 2*x+3
grad_of_y_wrt_x = tape.gradient(x,y)    

In [7]:
W = tf.Variable(tf.random.uniform((2,2))) # 가중치를 초기화
b = tf.Variable(tf.zeros((2,)))
x = tf.random.uniform((2,2))
with tf.GradientTape() as tape:
    y = tf.matmul(x,W) + b
grad_of_y_wrt_W_and_b = tape.gradient(y, [W,b])    

In [12]:
# 데이터 로드
from tensorflow.keras.datasets import mnist
(x_train,y_train),(x_test,y_test)  = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = x_train.reshape(x_train.shape[0],-1)
x_test = x_test.reshape(x_test.shape[0],-1)

In [13]:
x_train.shape, x_test.shape

((60000, 784), (10000, 784))

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

In [15]:
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy,
             metrics=['acc'])

In [16]:
model.fit(x_train,y_train,epochs=5,batch_size=128)

Epoch 1/5
469/469 [==============================] - 3s 5ms/step - loss: 0.2692 - acc: 0.9236
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1111 - acc: 0.9672
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0724 - acc: 0.9789
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0517 - acc: 0.9849
Epoch 5/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0388 - acc: 0.9885


In [17]:
# 텐서플로우를 이용해서 위의 식을 구현해 보기

In [80]:
import tensorflow as tf

class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):
        return [self.W, self.b]

In [81]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [82]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [83]:
#배치 제너레이터
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [84]:
# 훈련스텝 실행
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [85]:
# 가중치 업데이트

learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)

In [86]:
# 옵티마이져
from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [87]:
# 훈련
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"에포크 {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"{batch_counter}번째 배치 손실: {loss:.2f}")

In [88]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

에포크 0
0번째 배치 손실: 6.47
100번째 배치 손실: 2.22
200번째 배치 손실: 2.19
300번째 배치 손실: 2.08
400번째 배치 손실: 2.24
에포크 1
0번째 배치 손실: 1.88
100번째 배치 손실: 1.86
200번째 배치 손실: 1.81
300번째 배치 손실: 1.71
400번째 배치 손실: 1.85
에포크 2
0번째 배치 손실: 1.56
100번째 배치 손실: 1.56
200번째 배치 손실: 1.49
300번째 배치 손실: 1.41
400번째 배치 손실: 1.53
에포크 3
0번째 배치 손실: 1.30
100번째 배치 손실: 1.33
200번째 배치 손실: 1.23
300번째 배치 손실: 1.20
400번째 배치 손실: 1.29
에포크 4
0번째 배치 손실: 1.11
100번째 배치 손실: 1.15
200번째 배치 손실: 1.03
300번째 배치 손실: 1.04
400번째 배치 손실: 1.12
에포크 5
0번째 배치 손실: 0.97
100번째 배치 손실: 1.01
200번째 배치 손실: 0.89
300번째 배치 손실: 0.92
400번째 배치 손실: 1.00
에포크 6
0번째 배치 손실: 0.86
100번째 배치 손실: 0.91
200번째 배치 손실: 0.79
300번째 배치 손실: 0.83
400번째 배치 손실: 0.91
에포크 7
0번째 배치 손실: 0.78
100번째 배치 손실: 0.83
200번째 배치 손실: 0.71
300번째 배치 손실: 0.77
400번째 배치 손실: 0.84
에포크 8
0번째 배치 손실: 0.72
100번째 배치 손실: 0.76
200번째 배치 손실: 0.65
300번째 배치 손실: 0.71
400번째 배치 손실: 0.79
에포크 9
0번째 배치 손실: 0.67
100번째 배치 손실: 0.71
200번째 배치 손실: 0.60
300번째 배치 손실: 0.67
400번째 배치 손실: 0.74


In [89]:
# 모델평가하기
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"정확도: {matches.mean():.2f}")

정확도: 0.82
